## Теперь озвучка

In [17]:
# импортируем библиотеки
import requests
import uuid
import os
import random

# функция для получения токена
def get_token(auth_token, scope='SALUTE_SPEECH_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «SALUTE_SPEECH_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return None

In [14]:
# Считываем ключ авторизации из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('SBER_API_KEY'):
            SBER_API_KEY = line.split('=', 1)[1].strip(' \n\r')
            if not SBER_API_KEY.endswith('=='):  # Если нет ==, добавляем их
                SBER_API_KEY += '=='
            break

print(f"Прочитанное значение: '{SBER_API_KEY}'")
#'MTVjODUyZDMtZTU2Mi00ZWY5LWI3YzctZGY4YjUxZGMzMjhjOjllZmQ3NjZmLTVhYWQtNGYxOC1iN2I4LTQwOGMyY2IwZWI4Yg=='

response = get_token(SBER_API_KEY) # получаем токен
if response != None:
    salute_token = response.json()['access_token']

Прочитанное значение: 'MTVjODUyZDMtZTU2Mi00ZWY5LWI3YzctZGY4YjUxZGMzMjhjOjllZmQ3NjZmLTVhYWQtNGYxOC1iN2I4LTQwOGMyY2IwZWI4Yg=='


/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [19]:
# функция для синтеза речи
def synthesize_speech(text, token, format='wav16', voice='May_24000'):
    url = "https://smartspeech.sber.ru/rest/v1/text:synthesize" # URL для синтеза речи
    headers = {
        "Authorization": f"Bearer {token}", # токен авторизации
        "Content-Type": "application/text" # тип содержимого
    }
    params = {
        "format": format, # формат аудио
        "voice": voice # голос
    }
    response = requests.post(
            url, 
            headers=headers, 
            params=params, 
            data=text.encode(), # текст для синтеза
            verify=False # отключение проверки SSL
        )
    # Указание директории для аудиофайлов
    output_dir = '../audio'
    #os.makedirs(output_dir, exist_ok=True) #создание директории на случай, если она отсутствует

        # Генерация уникального имени файла (два варианта)
        # Вариант 1: Инкрементальное имя
    index = 1
    while os.path.exists(os.path.join(output_dir, f'output_{index}.wav')):
        index += 1
    output_file_increment = os.path.join(output_dir, f'output_{index}.wav')

        # Вариант 2: Случайное имя
    random_number = random.randint(1, 9999)
    output_file_random = os.path.join(output_dir, f'output_{random_number}.wav')

        # Выбор подхода: используем инкрементальный вариант для примера
    output_file = output_file_random

    # проверка на успешность синтеза
    if response.status_code == 200:
        # Сохранение синтезированного аудио в файл
        with open(output_file, 'wb') as f:
            f.write(response.content)
        print("Аудио успешно синтезировано и сохранено как 'output.wav'")

In [9]:
# =============================================================================
# ### Пример использования ###
# =============================================================================

recipe = """Из представленных ингредиентов можно приготовить коктейль "Привет от Дориана Грей Гуса".

Ингредиенты:
- 15 мл ежевичного ликера;
- 15 мл персикового шнапса;
- 15 мл светлого рома;
- 15 мл лимонного сока.

Приготовление:
1. Соединить все ингредиенты в шейкере.
2. Добавить лед.
3. Хорошо взбалтывать.
4. Процедить в шот.

Приятного аппетита!""" # Ваш текст здесь



In [21]:
# вызов функции для синтеза речи
synthesize_speech(recipe, salute_token)

/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Аудио успешно синтезировано и сохранено как 'output.wav'
